# Falcon

## Description

<p>Falcon (Fast-Fourier Lattice-based Compact Signatures) is a post-quantum digital signature algorithm based on lattice cryptography. Designed to be compact, efficient and secure versus quantum attacks. </p>

## Math model

### Signing

Given a message m, the signing process involves the following steps:

1. Generate a random value r.
2. Compute the hash H(m, r).
3. Solve the lattice problem to find a short vector s such that A * s = H(m, r) mod q .

The signature is the pair (s, r).

### Verification

Given a message  m and a signature (s, r), the verification process involves the following steps:

1. Compute the hash H(m, r).
2. Verify that A * s = H(m, r) mod q .

If the equation holds, the signature is valid.

In [11]:
import numpy as np
from numpy.fft import fft, ifft

# discrete Gaussian sampling
def discrete_gaussian_sample(n, sigma):
    return np.random.normal(0, sigma, n).astype(int)

# polynomial multiplication using FFT
def polynomial_multiply(a, b):
    return np.round(ifft(fft(a) * fft(b)).real).astype(int)

def falcon_keygen(n, sigma):
    # random polynomial f with coefficients in {-1, 0, 1}
    f = np.random.randint(-1, 2, n)
    
    # random polynomial g with coefficients sampled from a discrete Gaussian distribution
    g = discrete_gaussian_sample(n, sigma)
    
    # h = g / f in the ring Z[x]/(x^n + 1)
    h = polynomial_multiply(g, np.fft.ifft(1 / np.fft.fft(f)).real)
    
    # pub key is h, prv key is (f, g)
    return h, (f, g)


def falcon_sign(message, private_key, n, sigma):
    f, g = private_key
    
    # random polynomial y with coefficients sampled from a discrete Gaussian distribution
    y = discrete_gaussian_sample(n, sigma)
    
    # c = H(message) where H is a hash function
    c = np.sum(np.array([ord(char) for char in message])) % (2**32)
    
    # Compute z = y + c * f
    z = y + c * f
    
    return z, c

def falcon_verify(message, signature, public_key, n):
    z, c = signature
    h = public_key
    
    # c_prime = H(message)
    c_prime = np.sum(np.array([ord(char) for char in message])) % (2**32)
    
    return c == c_prime

if __name__ == "__main__":
    n = 512  # Degree of the polynomial
    sigma = 1.0  # Standard deviation for discrete Gaussian sampling

    # Key generation
    public_key, private_key = falcon_keygen(n, sigma)

    # Message to be signed
    message = "Hello, Falcon!"

    # Sign the message
    signature = falcon_sign(message, private_key, n, sigma)

    # Verify the signature
    is_valid = falcon_verify(message, signature, public_key, n)

    print("Signature is valid:", is_valid)

Signature is valid: True
